In [45]:
print("Ok")

Ok


In [46]:
%pwd

'C:\\Projects\\Medical_chatbot'

In [47]:
import os
os.chdir("C:\Projects\Medical_chatbot")

In [48]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [49]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [50]:
extracted_data = load_pdf_file(data = 'Data')

In [51]:
#Splitting the data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk

In [52]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6973


In [57]:
from langchain_huggingface import HuggingFaceEmbeddings

In [58]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [59]:
embeddings = download_hugging_face_embeddings()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [61]:
query_result = embeddings.embed_query("Hello World")
print("Lenght: ", len(query_result))

Lenght:  384


In [62]:
query_result

[-0.03447732329368591,
 0.031023161485791206,
 0.006734949070960283,
 0.02610895223915577,
 -0.03936203941702843,
 -0.16030243039131165,
 0.06692401319742203,
 -0.006441463716328144,
 -0.04745052009820938,
 0.014758840203285217,
 0.07087533921003342,
 0.055527545511722565,
 0.019193334504961967,
 -0.026251288130879402,
 -0.010109501890838146,
 -0.026940416544675827,
 0.022307397797703743,
 -0.022226659581065178,
 -0.1496926099061966,
 -0.017493031919002533,
 0.007676285691559315,
 0.05435223504900932,
 0.003254441311582923,
 0.031725917011499405,
 -0.08462145179510117,
 -0.029405971989035606,
 0.05159560590982437,
 0.048124030232429504,
 -0.003314853413030505,
 -0.058279167860746384,
 0.041969235986471176,
 0.022210724651813507,
 0.128188818693161,
 -0.022338958457112312,
 -0.011656282469630241,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122604131698608,
 -0.03117535263299942,
 0.0526995025575161,
 0.047034818679094315,
 -0.0842030942440033,
 -0.030056176707148552,
 -0.0207448

In [111]:
%pwd

'C:\\Projects\\Medical_chatbot'

In [112]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

True

In [113]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [115]:
print("Loaded Key:", os.getenv("PINECONE_API_KEY"))

Loaded Key: pcsk_7EjE1h_CMjGWcdqSfcEPU6vM3G9VM2RcBR6gGQgTvFSGcPR6Qj6vD7jZcCj9fs8pUQrmxt


In [116]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec, Pinecone
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension = 384,
        metric = "cosine",
        spec = ServerlessSpec(
            cloud = "aws",
            region = "us-east-1"
        )
    )

In [117]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [118]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [102]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings
)

In [103]:
docsearch

In [104]:
retriever = docsearch.as_retriever(search_type = 'similarity', search_kwargs = {"k": 3})

In [107]:
retrieved_docs = retriever.invoke("What is Acne")

In [108]:
retrieved_docs

[Document(id='127ccf5e-8f98-46d0-94b9-15d20b498d4e', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 298.0, 'page_label': '299', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteroids —A group of anti-inflammatory\nsubstances often used to treat skin conditions.\nImmune response—The protective reaction by the\nimmune system against foreign antigens (sub-\nstances that the body perceives as potentially dan-\ngerous). The immune system combats disease by\nneutralizing or destroying antigens.\ncontact dermatitis becomes a chronic and disabling con-\ndition that can have a profound effect on employability\nand quality of life.\nPrevention'),
 Document(id='e1d5f035-3679-4fd9-b113-0bb92ee97e78', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creat

In [119]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [122]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you"
    "don't know. Use three sentences maximum and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [123]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [124]:
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}